In [11]:
import pandas as pd
import numpy as np
import pickle
from itertools import product

In [4]:
raw_data = pd.read_pickle('./Data_Generation/df_pickle/question-context-20.pkl')
raw_data.reset_index(inplace=True)

In [25]:

# create a sample dataframe
#raw_data = pd.DataFrame({'question': ['What is your name?', 'What is your age?',"what is 2+3","what is 2+3",3],
#                   'context': ['My name is Stephen Hawking.', 'I am 76 years old.',5,"the number 5",3]})


# Assuming you have a DataFrame called df with columns 'question' and 'answer'
questions = raw_data['question'].unique()
answers = raw_data['context'].unique()

# Create a dictionary with question-answer pairs as keys and 1 as the value
original_pairs = {(row['question'], row['context']): 1 for _, row in raw_data.iterrows()}

# Generate all possible combinations and check if they are in the original_pairs dictionary
df = pd.DataFrame([(q, a, original_pairs.get((q, a), 0)) for q, a in product(questions, answers)], columns=['question', 'context', 'label'])

In [ ]:
df[]

In [33]:
test_q = np.random.choice(list(df['question'].unique()), int(len(df['question'].unique())*0.2))

array(['What is the density of a GMM with K components?',
       'What does the probability of four cylinders in the cars-dataset refer to?',
       'What is the important property of the Beta prior?',
       'How is the output of a neural network with L layers returned?',
       'What is the goal of K-means clustering?',
       'What is the relationship between entropy and the complexity of the distribution of a random variable?',
       'What is the intuition behind the entropy of a random, binary event?',
       'What is clustering in unsupervised learning?',
       'What is the significance of the rotation in the second dataset example?',
       'How does evidence affect the degree-of-belief in a proposition?',
       'What is the distribution used in Example 11.2.1?',
       'Why is visualization important in machine learning?',
       'What is the purity gain in decision trees?',
       'How is accuracy of a classifier calculated using the confusion matrix?',
       'What is the 

In [36]:
test = df[df['question'].isin(test_q)]
train = df[~df['question'].isin(test_q)]
train = pd.concat([train,train[train['label']==1].sample(n = len(train[train['label']==0]['label'])-len(train[train['label']==1]['label']), replace=True)], ignore_index = True)

Neural Net

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
neural_net = Sequential()
neural_net.add(Dense(256, input_dim=768*2, activation='relu'))
neural_net.add(Dense(256, activation='relu'))
neural_net.add(Dense(1, activation='sigmoid'))

neural_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])




In [ ]:
neural_net.fit(X_train, y_train,
          batch_size=128,
          epochs=2,
          verbose=1,
          validation_data=(X_test, y_test)